# Animación del Robot SCARA (usando análisis geométrico para la cinemática directa)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/oscar-ramos/fundamentos-robotica-python/blob/main/4-Cinematica-Directa/4-3-Metodo-Geometrico-RobotScara-Anim.ipynb)

Oscar E. Ramos Ponce, Universidad de Ingeniería y Tecnología - UTEC

Fundamentos de Robótica

### 1.&nbsp;Funciones preliminares

Se utilizará funciones de rotación y traslación definidas en `kinefunctions.py`. 

Si se trabaja en Google Colab, es necesario descargar el archivo `kinefunctions.py` a la máquina virtual con la que se está trabajando, lo que se realiza con la siguiente línea (wget):

In [ ]:
if 'google.colab' in str(get_ipython()):
    !wget https://raw.githubusercontent.com/oscar-ramos/fundamentos-robotica-python/main/4-Cinematica-Directa/kinefunctions.py

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from kinefunctions import Trotx, Troty, Trotz, Trasl


In [11]:
# Cinemática directa del robot
def cdirecta_scara(q, l1, l2, l3, l4):
    """ Retorna los sistemas de referencia de cada eslabón con respecto a la base
    """
    # Sistemas con respecto al sistema anterior
    T01 = Trasl(0.,0.,l1) @ Trotz(np.pi+q[0])
    T12 = Trasl(l2,0.,0.) @ Trotz(-np.pi/2+q[1])
    T23 = Trasl(l3,0.,0.)
    T34 = Trasl(0.,0.,-l4+q[2]) @ Trotz(np.pi/2+q[3])
    T4e = Trotx(np.pi)

    # Sistemas con respecto a la base
    T02 = T01 @ T12
    T03 = T02 @ T23
    T04 = T03 @ T34
    T0e = T04 @ T4e
    return T0e, (T01, T02, T03, T04)

In [12]:
# Gráfico del robot
def graph_scara(q, l1, l2, l3, l4, ax=None, k=0.4):
    """ Grafica el robot según la configuración articular. Las entradas son los valores articulares, 
    las longitudes de los eslabones y un factor para el tamaño con que se muetra los sistemas de referencia
    """
    # Cálculo de la cinemática directa
    Te, T = cdirecta_scara(q, l1, l2, l3, l4)
    # Borrar el gráfico
    if (ax==None):
        plt.clf()
        ax = plt.axes(projection='3d')
    # Nombres para los ejes
    ax.set_xlabel('x'); ax.set_ylabel('y'); ax.set_zlabel('z')
    # Transformaciones homogéneas con respecto a la base (ej. T2 es {2} con respecto a {0})
    T1 = T[0]; T2 = T[1]; T3 = T[2]; T4 = T[3]
    # Cuerpo del robot
    ax.plot([0, T1[0,3]], [0, T1[1,3]], [0, T1[2,3]], linewidth=3, color='k')
    ax.plot([T1[0,3], T2[0,3]], [T1[1,3], T2[1,3]], [T1[2,3], T2[2,3]], linewidth=3, color='k')
    ax.plot([T2[0,3], T3[0,3]], [T2[1,3], T3[1,3]], [T2[2,3], T3[2,3]], linewidth=3, color='k')
    ax.plot([T3[0,3], T4[0,3]], [T3[1,3], T4[1,3]], [T3[2,3], T4[2,3]], linewidth=3, color='k')
    # Puntos en las articulaciones
    ax.scatter(0, 0, 0, color='g', s=50)
    # "Cilindros" para representar la dirección de las articulaciones
    ax.plot([T1[0,3], T1[0,3]], [T1[1,3], T1[1,3]], [T1[2,3]-0.1, T1[2,3]+0.1], linewidth=10, color='g')
    ax.plot([T2[0,3], T2[0,3]], [T2[1,3], T2[1,3]], [T2[2,3]-0.1, T2[2,3]+0.1], linewidth=10, color='g')
    ax.plot([T3[0,3], T3[0,3]], [T3[1,3], T3[1,3]], [T3[2,3]-0.1, T3[2,3]+0.1], linewidth=10, color='g') 
    ax.plot([T4[0,3], T4[0,3]], [T4[1,3], T4[1,3]], [T4[2,3]-0.05, T4[2,3]+0.05], linewidth=10, color='g')    
    # Efector final (definido por 4 puntos)
    p1 = np.array([0, 0.1, 0, 1]); p2 = np.array([0, 0.1, 0.2, 1])
    p3 = np.array([0, -0.1, 0, 1]); p4 = np.array([0, -0.1, 0.2, 1])
    p1 = Te.dot(p1); p2 = Te.dot(p2); p3 = Te.dot(p3); p4 = Te.dot(p4)
    # Sistema de referencia del efector final (con respecto al sistema 0)
    ax.plot([Te[0,3],Te[0,3]+k*Te[0,0]], [Te[1,3],Te[1,3]+k*Te[1,0]], [Te[2,3],Te[2,3]+k*Te[2,0]], color='r')
    ax.plot([Te[0,3],Te[0,3]+k*Te[0,1]], [Te[1,3],Te[1,3]+k*Te[1,1]], [Te[2,3],Te[2,3]+k*Te[2,1]], color='g')
    ax.plot([Te[0,3],Te[0,3]+k*Te[0,2]], [Te[1,3],Te[1,3]+k*Te[1,2]], [Te[2,3],Te[2,3]+k*Te[2,2]], color='b')
    # Sistema de referencia de la base (0)
    ax.plot([0,k], [0,0], [0,0], color='r')
    ax.plot([0,0], [0,k], [0,0], color='g')
    ax.plot([0,0], [0,0], [0,k], color='b')
    # Gráfico del efector final
    ax.plot([p1[0],p2[0]], [p1[1],p2[1]], [p1[2],p2[2]], color='b', linewidth=3)
    ax.plot([p3[0],p4[0]], [p3[1],p4[1]], [p3[2],p4[2]], color='b', linewidth=3)
    ax.plot([p1[0],p3[0]], [p1[1],p3[1]], [p1[2],p3[2]], color='b', linewidth=3)
    # Punto de vista
    ax.view_init(elev=25, azim=45)
    # Límites para los ejes
    ax.set_xlim3d(-2, 2)
    ax.set_ylim3d(-2, 2)
    ax.set_zlim3d(0,1.2)

### 2.&nbsp;Animación en Modo Local

Se realizará la animación de varias configuraciones articulares, lo cual simula el movimiento del robot.

Nota: la siguiente línea (qt) no funcionará en Google Colab. Para Colab ejecutar la sección 3.

In [13]:
# "qt" realiza el gráfico en una ventana nueva
%matplotlib qt

In [15]:
l1 = 1.0; l2 = 1.0; l3 = 1.0; l4 = 0.5

# Se abrirá una nueva ventana donde se visualizará el robot
for i in range(60):
    q = [np.deg2rad(i), np.deg2rad(0.5*i), 0.005*i, np.deg2rad(0.5*i)]    # En grados
    graph_scara(q, l1, l2, l3, l4)

    plt.pause(0.005)   

### 3.&nbsp;Animación en Google Colab

En Google Colab ejecutar la siguiente línea para instalar el paquete `celluloid`.

In [9]:
if 'google.colab' in str(get_ipython()):
    !pip install celluloid --quiet

In [ ]:
from IPython.display import HTML
from celluloid import Camera

In [ ]:
ax = plt.axes(projection='3d')
fig = ax.figure

camera = Camera(fig)

# Longitudes del robot
l1 = 1.0; l2 = 1.0; l3 = 1.0; l4 = 0.5
# Bucle de simulación
for i in range(60):
    q = [np.deg2rad(i), np.deg2rad(0.5*i), 0.005*i, np.deg2rad(0.5*i)]    # En grados
    graph_scara(q, l1, l2, l3, l4, ax)
    camera.snap()

plt.close()

In [ ]:
# Crear la animación
anim = camera.animate(interval = 40, repeat = True, repeat_delay = 500)

# Mostrar el resultado
HTML(anim.to_html5_video())